In [14]:
import pandas as pd
import pywt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
#torch.backends.cudnn.benchmark = True

## Wavelet transform

In [15]:
def van_haar(data):
    ncol = data.shape[1]
    nrow = data.shape[0]
    for i in range(ncol):
        cur_col = data[:,i].copy()
        (cA, cD) = pywt.dwt(cur_col, 'haar')
        new_col = np.reshape(np.concatenate((cA,cD), 0),(nrow,1))
        data = np.hstack((data,new_col))
    data = data.reshape(nrow,-1)
    return data

#van_haar(test.to_numpy()).shape

## PyTorch

In [16]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(
            self, 
            n_inputs, 
            n_outputs, 
            kernel_size, 
            stride,
            dilation,
            padding,
            dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(
            nn.Conv1d(
                n_inputs,
                n_outputs,
                kernel_size,
                stride=stride,
                padding=padding,
                dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(
            nn.Conv1d(
                n_outputs,
                n_outputs,
                kernel_size,
                stride=stride,
                padding=padding,
                dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)


        self.net = nn.Sequential(
            self.conv1,
            self.chomp1,
            self.relu1,
            self.dropout1,
            self.conv2,
            self.chomp2,
            self.relu2,
            self.dropout2)
        
        self.downsample = nn.Conv1d(
            n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, 
                                     out_channels, 
                                     kernel_size, 
                                     stride=1, 
                                     dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, 
                                     dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [17]:
def reference_transform(tensor):
    array = tensor.numpy()
    out1, out2= pywt.dwt(array, "haar")
    out1 = torch.from_numpy(out1)
    out2 = torch.from_numpy(out2)
    
    #concatenate each channel to be able to concatenate it to the untransformed data
    #everything will then be split when fed to the network
    return torch.cat((out1, out2),-1)
    

In [18]:
class WaveletPart(nn.Module):

    def __init__(self,input_length, input_size, output_size):
        super(WaveletPart, self).__init__()

        # used two different layers here as in the paper but in the github code, they are the same
        self.fc1 = nn.Linear(input_size, output_size)
        self.fc2 = nn.Linear(input_size, output_size)

        self.input_size = input_size
        self.input_length = input_length

        self.haar = reference_transform

    def init_weight(self):
        self.fc1.weight.data.normal_(0, 0.01)
        self.fc1.bias.data.normal_(0, 0.01)
        self.fc2.weight.data.normal_(0, 0.01)
        self.fc2.bias.data.normal_(0, 0.01)

    def forward(self, x):
        # split the wavelet transformed data along third dim
        x1, x2 = torch.split(x, self.input_length//2, 2)

        # reshape everything to feed to the linear layer
        bsize = x.size()[0]
        x1 = self.fc1(x1.reshape((bsize, -1, 1)).squeeze())
        x2 = self.fc2(x2.reshape((bsize, -1, 1)).squeeze())
        x1 = x1.reshape(bsize, -1)
        x2 = x2.reshape(bsize, -1)
        return torch.cat((x1,x2),-1)

In [19]:
class Driver2Vec(nn.Module):
    def __init__(
            self, 
            input_size, 
            input_length, 
            num_channels,
            output_size, 
            kernel_size, 
            dropout,
            do_wavelet = True,
            fc_output_size = 15):
        super(Driver2Vec, self).__init__()
        
        self.tcn = TemporalConvNet(input_size, 
                                   num_channels, 
                                   kernel_size=kernel_size, 
                                   dropout=dropout)
        self.wavelet = do_wavelet
        if self.wavelet:
            self.haar = WaveletPart(input_length, input_size*input_length//2, fc_output_size)

            linear_size = num_channels[-1] + fc_output_size*2
        else: 
            linear_size = num_channels[-1]
        self.input_length = input_length

        self.input_bn = nn.LayerNorm(linear_size)
        self.linear = nn.Linear( linear_size, output_size)

    def forward(self, inputs, print_temp = False):
        """Inputs have to have dimension (N, C_in, L_in*2)
        the base time series, and the two wavelet transform channel are concatenated along dim2"""
        
        # split the inputs, in the last dim, first is the unchanged data, then 
        # the wavelet transformed data
        input_tcn, input_haar = torch.split(inputs, self.input_length, 2)

        # feed each one to their corresponding network
        y1 = self.tcn(input_tcn)
        y1 = y1[:,:,-1] # for the TCN, only the last output element interests us

        if self.wavelet:
            y2 = self.haar(input_haar)

            out = torch.cat((y1,y2),1)
            bsize = out.shape[0]

        if bsize > 0: # issue when the batch size is 1, can't batch normalize it
            out = self.input_bn(out)
        else:
            out = out
        out = self.linear(out)

        if print_temp:
            print(out)
        
        return out

In [20]:
import os

def preprocess(df):
    return (
        df.drop(
            ["FOG",
             "FOG_LIGHTS",
             "FRONT_WIPERS",
             "HEAD_LIGHTS",
             "RAIN",
             "REAR_WIPERS",
             "SNOW",
            ], axis=1
        )
    )

def load_dataset(input_dir):
    x=[]
    y=[]
    for dir,_,files in os.walk(input_dir):
        for file in files:
            label = int(file.split("_")[1])
            df = pd.read_csv(dir + "/" + file, index_col=0)
            df = preprocess(df).to_numpy().transpose()
            x.append(torch.from_numpy(df).float())
            y.append(label)
    return x,y
        
class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, input_dir, input_length=500, max_length = 1000):
        'Initialization'
        self.data, self.labels = load_dataset(input_dir)
        self.index = [i for i in range(len(self.data))]
        self.length = input_length
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        start_pos = int(np.random.uniform(0,self.max_length - self.length+1))
        X_anchor = self.data[index][:,start_pos:start_pos+self.length]
        anchor_wvlt = reference_transform(X_anchor)
        y_anchor = self.labels[index]

        # create list of possible positive index samples
        positive_list = [
            i for i in self.index if self.labels[i] == y_anchor and i != index]
        positive_index = np.random.choice(positive_list)
        positive = self.data[positive_index][:,start_pos:start_pos+self.length]
        # get the wavelet transform of that sample (the two channels are concatenated along the last dim)
        positive_wvlt = reference_transform(positive)
        
        # same here
        negative_list = [
            i for i in self.index if self.labels[i] != y_anchor]
        negative_index = np.random.choice(negative_list)
        negative = self.data[negative_index][:,start_pos:start_pos+self.length]
        negative_wvlt = reference_transform(negative)

        # concatenate the data for the TCN and the haar wavelet transform
        # they will be split in the forward pass
        return torch.cat((X_anchor, anchor_wvlt),1), \
            torch.cat((positive, positive_wvlt),1), \
            torch.cat((negative, negative_wvlt),1), \
            y_anchor


In [21]:
input_channels = 31
input_length = 400
channel_sizes = [32]
output_size = 62
kernel_size = 16
dropout = 0.1
model = Driver2Vec(input_channels, input_length, channel_sizes, output_size, kernel_size=kernel_size, dropout=dropout)
model.to(device)

Driver2Vec(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(31, 32, kernel_size=(16,), stride=(1,), padding=(15,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.1, inplace=False)
        (conv2): Conv1d(32, 32, kernel_size=(16,), stride=(1,), padding=(15,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.1, inplace=False)
        (net): Sequential(
          (0): Conv1d(31, 32, kernel_size=(16,), stride=(1,), padding=(15,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.1, inplace=False)
          (4): Conv1d(32, 32, kernel_size=(16,), stride=(1,), padding=(15,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.1, inplace=False)
        )
        (downsample): Conv1d(31, 32, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
    )
  )
  (haar): WaveletPart(
    (fc1): Linear(in_features=6200, out_

In [22]:

# datasets parameters
params = {'batch_size': 4,
          'shuffle': True,
          'num_workers': 1}
epochs = 50

training_set = Dataset("./dataset", input_length)
training_generator = DataLoader(training_set, **params)

loss = nn.TripletMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [23]:
model.train()
for epoch in (pbar := tqdm(range(epochs))):
    loss_list = []
    for anchor, positive, negative, label in training_generator:
        print(label)
        anchor = anchor.to(device)
        positive = positive.to(device)
        negative = negative.to(device)

        y_anchor = model(anchor)
        y_positive = model(positive)
        y_negative = model(negative)

        loss_value = loss(y_anchor, y_positive, y_negative)
        loss_value.backward()

        optimizer.step()

        loss_list.append(loss_value.cpu().detach().numpy())
    print(anchor.cpu())
    #print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, epochs, np.mean(loss_list)))
    pbar.set_description("Loss: %0.5g, Epochs" % np.mean(loss_list))



  0%|          | 0/50 [00:00<?, ?it/s]

tensor([1, 5, 3, 5])
tensor([4, 1, 2, 3])
tensor([5, 2, 4, 4])
tensor([2, 3, 5, 1])
tensor([4, 2, 1])
tensor([[[-3.0267e-03, -2.8686e-03, -2.7900e-03,  ..., -2.2065e-02,
          -1.3372e-02,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.6573e-03,
           0.0000e+00,  0.0000e+00],
         [-1.4688e-05, -2.2472e-05, -7.7614e-05,  ...,  4.9613e-04,
           1.4338e-03,  0.0000e+00],
         ...,
         [ 5.8618e-07, -3.1738e-07, -5.7836e-07,  ..., -3.3724e-04,
          -1.9719e-04,  0.0000e+00],
         [-1.6263e-03, -1.5005e-03, -1.4547e-03,  ...,  7.8996e-04,
           6.1341e-04,  0.0000e+00],
         [ 5.3307e-02,  5.3116e-02,  5.2924e-02,  ..., -1.6271e-03,
          -2.7118e-04,  0.0000e+00]],

        [[ 3.0527e-05,  2.9340e-05,  2.9340e-05,  ...,  0.0000e+00,
           1.7452e-06,  4.1163e-07],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-4.2323e-05, -2.3913e-05, -2.391

In [24]:
params = {'batch_size': 1,
          'shuffle': False,
          'num_workers': 1}

classifier_set = Dataset("./dataset")
classifier_generator = DataLoader(training_set, **params)

x_train = []
y_train = []
for data,_, _, label in classifier_generator:

    data = data.to(device)
    embed = model(data, True)

    y = np.zeros(5, dtype=int)
    y[label-1] = 1

    x_train.append(embed.cpu().detach().numpy().squeeze())
    y_train.append(int(label)-1)

x_train = np.array(x_train)
y_train = np.array(y_train)

tensor([[-0.3865,  0.9810,  0.1003, -0.3639, -0.2752, -0.6231, -1.2523,  0.2532,
          0.0443,  0.4893,  0.3850, -0.2523,  0.4911,  0.4810,  0.3517, -0.6362,
         -0.3364, -0.2467,  0.2867, -0.6748,  0.1772, -0.0222,  0.3622,  0.5863,
         -0.3989,  1.2377, -0.4959, -1.1321, -0.5391, -0.0677,  0.4520, -0.3202,
         -0.6256,  0.0466, -0.1363,  0.9193,  0.2105, -0.2397,  0.1133,  0.0448,
         -0.0974, -0.2640, -0.1472,  1.1977, -0.3289,  0.3234,  0.0305, -0.4230,
          0.4999, -0.3798,  0.6320, -0.2215, -0.1204,  0.7319,  0.7176, -1.0795,
          0.2814, -0.7980, -0.9470,  0.3441, -0.2258, -0.0268]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-2.6875e-01,  1.0655e+00, -1.1850e-02, -5.5439e-01, -2.5061e-01,
         -4.4231e-01, -1.3891e+00,  7.9240e-02, -2.1918e-02,  3.9837e-01,
          5.2587e-01, -1.3702e-01,  3.7074e-01,  6.6749e-01,  3.6658e-01,
         -7.4350e-01, -3.4653e-01,  1.4632e-04,  1.2112e-01, -7.4504e-01,
          6.3660e-02, 

In [27]:
import lightgbm as lgbm

params = {'batch_size': 1,
          'shuffle': False,
          'num_workers': 1}

classifier_set = Dataset("./dataset")
classifier_generator = DataLoader(training_set, **params)

x_train = []
y_train = []
for data,_, _, label in classifier_generator:

    data = data.to(device)
    embed = model(data)

    y = np.zeros(5, dtype=int)
    y[label-1] = 1

    x_train.append(embed.cpu().detach().numpy().squeeze())
    y_train.append(int(label)-1)

x_train = np.array(x_train)
y_train = np.array(y_train)

lgb_train = lgbm.Dataset(x_train, y_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':5,
    'metric': 'multi_logloss',
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'max_depth': 12,
    'num_trees': 100,
    'verbose': 0,
    'min_data_in_leaf': 2 # May need to change that with a real test set
}

clf = lgbm.train(params,lgb_train)

/home/wslachille/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000353 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [28]:
y_pred = clf.predict(x_train)
print(y_pred)

[[9.99818144e-01 5.63033417e-05 1.92504551e-05 6.63249503e-05
  3.99771877e-05]
 [9.99763370e-01 6.11060201e-05 3.31994620e-05 1.05544602e-04
  3.67801542e-05]
 [9.99771343e-01 3.22567206e-05 5.52291360e-05 8.54713853e-05
  5.56995282e-05]
 [9.99859042e-01 3.20171856e-05 3.20830287e-05 3.23620108e-05
  4.44954241e-05]
 [3.15769012e-05 9.99856831e-01 3.42646313e-05 2.09011273e-05
  5.64264668e-05]
 [3.92524333e-05 9.99825307e-01 4.46546488e-05 4.49671352e-05
  4.58192660e-05]
 [2.54924887e-05 9.99825267e-01 2.24281553e-05 9.73826770e-05
  2.94296399e-05]
 [5.24502303e-05 9.99816381e-01 3.42561936e-05 5.88065806e-05
  3.81056861e-05]
 [6.34040443e-05 7.26181095e-05 9.99733232e-01 7.09769389e-05
  5.97686070e-05]
 [4.71246023e-05 8.03120679e-05 9.99743565e-01 2.22260625e-05
  1.06772601e-04]
 [1.54402235e-04 4.53038385e-05 9.99714581e-01 9.34460135e-06
  7.63680106e-05]
 [6.89404340e-05 5.85121066e-05 4.94006112e-05 9.99772257e-01
  5.08895955e-05]
 [3.05850295e-05 4.63406442e-05 2.511034